[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ruliana/pytorch-katas/blob/main/dan_1/kata_03_he_ao_world_accident_pattern_detector_unrevised.ipynb)

## 🏮 The Ancient Scroll Unfurls 🏮

**THE MYSTERY OF CONVENIENT CLUMSINESS: DETECTING HIDDEN PATTERNS IN APPARENT ACCIDENTS**

Dan Level: 1 (Temple Sweeper) | Time: 45 minutes | Sacred Arts: Multi-variable Classification, Binary Cross-Entropy, Feature Engineering

## 📜 THE CHALLENGE

Master Ao-Tougrad sits in the temple's shadowy corner, watching He-Ao-World apologetically clean up yet another spilled ink incident that occurred exactly when a difficult scroll needed "accidental" correction. "Young grasshopper," the master whispers, "observe how the temple janitor's daily mishaps follow curious patterns. Some accidents stem from genuine clumsiness of aged hands, but others... others happen with suspiciously perfect timing. The wise observer notices that time of day, location, presence of witnesses, and task complexity all whisper secrets about the true nature of each incident."

He-Ao-World shuffles by, bumping into a meditation cushion at precisely the moment when a particularly boring sutra needs interrupting, then glances around nervously before apologizing profusely. Master Ao-Tougrad's eyes glint with knowing amusement. "Your task, grasshopper, is to build a sacred classifier that can distinguish between innocent fumbles and masterfully timed 'accidents.' Train your neural network to recognize the hidden patterns using multiple clues simultaneously - for the truth lies not in any single observation, but in the convergence of many seemingly unrelated details."

## 🎯 THE SACRED OBJECTIVES

- [ ] Master multi-variable input processing with PyTorch tensors
- [ ] Implement binary classification using multiple features simultaneously  
- [ ] Learn to handle different types of input data (categorical and continuous)
- [ ] Practice proper gradient flow management in multi-feature models
- [ ] Understand how multiple inputs combine to influence predictions
- [ ] Build intuition for feature importance in classification decisions

In [ ]:
# 📦 FIRST CELL - ALL IMPORTS AND CONFIGURATION
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
from typing import Tuple

# Set reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Global configuration constants
DEFAULT_CHAOS_LEVEL = 0.1
SACRED_SEED = 42
N_FEATURES = 4  # time_of_day, location, witnesses, task_complexity

In [ ]:
# 🕵️ THE SACRED ACCIDENT OBSERVATION SCROLL

def generate_he_ao_accident_data(n_observations: int = 200, chaos_level: float = 0.1, 
                                sacred_seed: int = 42) -> Tuple[torch.Tensor, torch.Tensor]:
    """
    Generate observations of He-Ao-World's daily incidents around the temple.
    
    Ancient wisdom suggests that suspicious timing follows these patterns:
    - Morning hours (6-9 AM): 70% more likely to be suspicious (masters need interruptions)
    - Sacred hall locations: 60% more likely to be suspicious (important ceremonies)
    - No witnesses present: 80% more likely to be suspicious (perfect cover)
    - Complex tasks: 90% more likely to be suspicious (creates better distractions)
    
    Features:
    0. time_of_day: Hour of day (0-23)
    1. location: Temple location (0=courtyard, 1=sacred_hall, 2=kitchen, 3=library)
    2. witnesses: Number of witnesses present (0-5)
    3. task_complexity: Complexity of nearby task (0.0-1.0)
    
    Args:
        n_observations: Number of accidents to simulate
        chaos_level: Amount of randomness in the patterns (0.0 = perfectly predictable, 1.0 = pure chaos)
        sacred_seed: Ensures consistent randomness for reproducible wisdom
        
    Returns:
        Tuple of (features, is_suspicious) as sacred tensors
        features shape: (n_observations, 4)
        is_suspicious shape: (n_observations, 1) - 1 for suspicious, 0 for genuine
    """
    torch.manual_seed(sacred_seed)
    np.random.seed(sacred_seed)
    
    # Generate random features
    time_of_day = torch.randint(0, 24, (n_observations,)).float()  # 0-23 hours
    location = torch.randint(0, 4, (n_observations,)).float()      # 0-3 locations  
    witnesses = torch.randint(0, 6, (n_observations,)).float()     # 0-5 witnesses
    task_complexity = torch.rand(n_observations)                   # 0.0-1.0 complexity
    
    # Stack features together
    features = torch.stack([time_of_day, location, witnesses, task_complexity], dim=1)
    
    # Calculate suspicion probability based on He-Ao-World's hidden patterns
    base_suspicion = 0.3  # 30% base chance of being suspicious
    
    # Morning hours (6-9 AM) increase suspicion by 0.4
    morning_bonus = torch.where((time_of_day >= 6) & (time_of_day <= 9), 0.4, 0.0)
    
    # Sacred hall (location=1) increases suspicion by 0.3  
    location_bonus = torch.where(location == 1, 0.3, 0.0)
    
    # No witnesses (witnesses=0) increases suspicion by 0.5
    witness_bonus = torch.where(witnesses == 0, 0.5, 0.0)
    
    # High task complexity increases suspicion
    complexity_bonus = task_complexity * 0.4  # Up to 0.4 bonus
    
    # Calculate final suspicion probability
    suspicion_prob = base_suspicion + morning_bonus + location_bonus + witness_bonus + complexity_bonus
    
    # Add chaos to make the pattern learnable but not obvious
    chaos = torch.randn(n_observations) * chaos_level * 0.2
    suspicion_prob = torch.clamp(suspicion_prob + chaos, 0.0, 1.0)
    
    # Generate binary labels based on probability
    is_suspicious = torch.bernoulli(suspicion_prob).unsqueeze(1)
    
    return features, is_suspicious

def visualize_accident_patterns(features: torch.Tensor, is_suspicious: torch.Tensor):
    """Reveal the hidden patterns in He-Ao-World's accidents."""
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    
    genuine_mask = (is_suspicious.squeeze() == 0)
    suspicious_mask = (is_suspicious.squeeze() == 1)
    
    feature_names = ['Time of Day (Hour)', 'Location', 'Witnesses Present', 'Task Complexity']
    location_labels = ['Courtyard', 'Sacred Hall', 'Kitchen', 'Library']
    
    for i, (ax, feature_name) in enumerate(zip(axes.flat, feature_names)):
        genuine_data = features[genuine_mask, i]
        suspicious_data = features[suspicious_mask, i]
        
        if i == 1:  # Location - use bar chart
            locations = [0, 1, 2, 3]
            genuine_counts = [(genuine_data == loc).sum().item() for loc in locations]
            suspicious_counts = [(suspicious_data == loc).sum().item() for loc in locations]
            
            x = np.arange(len(locations))
            width = 0.35
            
            ax.bar(x - width/2, genuine_counts, width, label='Genuine Accidents', 
                   color='lightblue', alpha=0.7)
            ax.bar(x + width/2, suspicious_counts, width, label='Suspicious Timing', 
                   color='red', alpha=0.7)
            ax.set_xticks(x)
            ax.set_xticklabels(location_labels)
            ax.set_ylabel('Count')
        else:
            ax.hist(genuine_data.numpy(), bins=20, alpha=0.6, label='Genuine Accidents', 
                   color='lightblue', density=True)
            ax.hist(suspicious_data.numpy(), bins=20, alpha=0.6, label='Suspicious Timing', 
                   color='red', density=True)
            ax.set_ylabel('Density')
        
        ax.set_title(f'{feature_name} Distribution')
        ax.set_xlabel(feature_name)
        ax.legend()
        ax.grid(True, alpha=0.3)
    
    plt.suptitle("He-Ao-World's Accident Patterns: Genuine vs Suspicious", fontsize=16)
    plt.tight_layout()
    plt.show()

def visualize_predictions(features: torch.Tensor, target: torch.Tensor, 
                         predictions: torch.Tensor = None):
    """Display Master Ao-Tougrad's assessment of your pattern recognition."""
    plt.figure(figsize=(12, 8))
    
    # Create scatter plot using time vs complexity (most informative features)
    genuine_mask = (target.squeeze() == 0)
    suspicious_mask = (target.squeeze() == 1)
    
    plt.scatter(features[genuine_mask, 0], features[genuine_mask, 3], 
               c='lightblue', alpha=0.6, s=50, label='Genuine Accidents')
    plt.scatter(features[suspicious_mask, 0], features[suspicious_mask, 3], 
               c='red', alpha=0.6, s=50, label='Suspicious Timing')
    
    if predictions is not None:
        # Show prediction confidence as background colors
        pred_probs = torch.sigmoid(predictions).detach().numpy().squeeze()
        
        # Create prediction boundary visualization
        xx = np.linspace(0, 23, 50)
        yy = np.linspace(0, 1, 50)
        XX, YY = np.meshgrid(xx, yy)
        
        plt.contour(XX, YY, np.ones_like(XX) * 0.5, levels=[0.5], 
                   colors='gold', linewidths=3, linestyles='--',
                   label='Decision Boundary (50%)')
    
    plt.xlabel('Time of Day (Hour)')
    plt.ylabel('Task Complexity')
    plt.title('The Hidden Patterns in He-Ao-World\'s Accidents\n(Time vs Complexity View)')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.xlim(0, 23)
    plt.ylim(0, 1)
    
    # Add annotations for key insights
    plt.axvspan(6, 9, alpha=0.1, color='orange', label='Morning Hours (High Suspicion)')
    plt.axhspan(0.7, 1.0, alpha=0.1, color='orange', label='Complex Tasks (High Suspicion)')
    
    plt.show()

## 🧠 THE SACRED NEURAL NETWORK TEMPLE

In [ ]:
# 🕵️ FIRST MOVEMENTS - THE PATTERN DETECTION ARTIFACT

class AccidentPatternDetector(nn.Module):
    """A mystical artifact for detecting hidden patterns in apparent clumsiness."""
    
    def __init__(self, input_features: int = 4):
        super(AccidentPatternDetector, self).__init__()
        # TODO: Create the Linear layer for multi-variable classification
        # Hint: input_features=4 (time, location, witnesses, complexity) -> output=1 (suspicious probability)
        self.linear = None
        
        # TODO: Add sigmoid activation for probability output
        # Hint: nn.Sigmoid() converts raw outputs to probabilities (0-1)
        self.sigmoid = None
    
    def forward(self, features: torch.Tensor) -> torch.Tensor:
        """Channel multiple clues through the mystical detection network."""
        # TODO: Pass features through linear layer
        raw_output = None
        
        # TODO: Apply sigmoid to get probability of suspicious timing
        # Remember: probabilities should be between 0 and 1
        probability = None
        
        return probability

def train_detector(model: nn.Module, features: torch.Tensor, target: torch.Tensor,
                  epochs: int = 1500, learning_rate: float = 0.01) -> list:
    """
    Train the accident pattern detection model.
    
    Returns:
        List of loss values during training
    """
    # TODO: Choose your loss calculation method for binary classification
    # Hint: Binary Cross Entropy Loss is the sacred choice for yes/no predictions
    criterion = None
    
    # TODO: Choose your parameter updating method  
    # Hint: SGD remains the traditional path for classification
    optimizer = None
    
    losses = []
    accuracies = []
    
    for epoch in range(epochs):
        # TODO: CRITICAL - Clear the gradient spirits from previous cycle
        # Hint: The spirits accumulate if not banished properly
        
        # TODO: Forward pass - get probability predictions
        predictions = None
        
        # TODO: Compute the binary classification loss
        loss = None
        
        # TODO: Backward pass - compute gradients
        
        # TODO: Update parameters
        
        # Calculate accuracy for monitoring
        with torch.no_grad():
            predicted_labels = (predictions > 0.5).float()
            accuracy = (predicted_labels == target).float().mean().item()
        
        losses.append(loss.item())
        accuracies.append(accuracy)
        
        # Report progress to Master Ao-Tougrad
        if (epoch + 1) % 200 == 0:
            print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}, Accuracy: {accuracy:.3f}')
            if accuracy > 0.75:
                print("🔮 Master Ao-Tougrad whispers: 'The patterns become visible to you...'")
    
    return losses, accuracies

## ⚡ THE TRIALS OF MASTERY

### Trial 1: Basic Pattern Recognition
- [ ] Loss decreases consistently (the hidden patterns reveal themselves)
- [ ] Final loss below 0.6 (Master Ao-Tougrad approves your perception)
- [ ] Training accuracy above 75% (you detect most suspicious incidents)
- [ ] Model learns meaningful weights for each feature (morning hours, sacred hall, etc.)

### Trial 2: Understanding Test

In [ ]:
def test_your_wisdom(model, features, target):
    """Master Ao-Tougrad's evaluation of your pattern detection mastery."""
    model.eval()
    
    # Test 1: Model produces correct output shapes
    test_features = torch.tensor([[8.0, 1.0, 0.0, 0.9],   # Morning, sacred hall, no witnesses, complex task
                                 [14.0, 0.0, 3.0, 0.1],   # Afternoon, courtyard, witnesses, simple task
                                 [22.0, 2.0, 1.0, 0.5]])  # Evening, kitchen, one witness, medium task
    
    predictions = model(test_features)
    assert predictions.shape == (3, 1), f"Shape mismatch: expected (3, 1), got {predictions.shape}"
    
    # Test 2: Probabilities are in valid range
    assert torch.all(predictions >= 0) and torch.all(predictions <= 1), "Predictions must be probabilities (0-1)!"
    
    # Test 3: Logical pattern recognition
    morning_sacred_complex = predictions[0].item()  # Should be highly suspicious
    afternoon_courtyard_simple = predictions[1].item()  # Should be less suspicious
    
    print(f"Morning sacred hall incident: {morning_sacred_complex:.3f} suspicion")
    print(f"Afternoon courtyard incident: {afternoon_courtyard_simple:.3f} suspicion")
    
    # Test 4: Model performance on training data
    with torch.no_grad():
        all_predictions = model(features)
        predicted_labels = (all_predictions > 0.5).float()
        accuracy = (predicted_labels == target).float().mean().item()
        
        print(f"Overall detection accuracy: {accuracy:.3f}")
        
        # Calculate precision and recall for suspicious incidents
        true_positives = ((predicted_labels == 1) & (target == 1)).sum().item()
        false_positives = ((predicted_labels == 1) & (target == 0)).sum().item()
        false_negatives = ((predicted_labels == 0) & (target == 1)).sum().item()
        
        precision = true_positives / (true_positives + false_positives + 1e-8)
        recall = true_positives / (true_positives + false_negatives + 1e-8)
        
        print(f"Precision (suspicious predictions that are correct): {precision:.3f}")
        print(f"Recall (suspicious incidents detected): {recall:.3f}")
    
    if accuracy > 0.75 and morning_sacred_complex > afternoon_courtyard_simple:
        print("\n🎉 Master Ao-Tougrad emerges from the shadows with approval!")
        print("   'Your eyes now perceive the subtle patterns that others miss.'")
        return True
    else:
        print("\n🤔 Master Ao-Tougrad remains silent. The patterns elude you still...")
        return False

## 🌸 THE FOUR PATHS OF MASTERY: PROGRESSIVE EXTENSIONS

### Extension 1: Cook Oh-Pai-Timizer's Kitchen Timing Analysis
*"The best meals require precise timing - just like the best 'accidents'!"*

*Cook Oh-Pai-Timizer wipes flour-covered hands on an apron while stirring a bubbling pot*

"Grasshopper! I notice you've learned to spot He-Ao-World's suspicious timing. But have you considered the deeper patterns? In my kitchen, I know that some combinations of ingredients create stronger flavors than others. Perhaps certain combinations of features create stronger suspicion signals too?"

**NEW CONCEPTS**: Feature interactions, feature engineering, polynomial features  
**DIFFICULTY**: +15% (still Dan 1, but with feature combinations)

In [ ]:
def create_feature_interactions(features: torch.Tensor) -> torch.Tensor:
    """
    Create interaction features like a master chef combines ingredients.
    
    Original features: [time, location, witnesses, complexity]
    Add interactions: time*complexity, location*witnesses, time*location, etc.
    
    Returns:
        Enhanced feature tensor with original + interaction features
    """
    # TODO: Create meaningful feature interactions
    # Hint: torch.cat() can combine tensors along a dimension
    # Hint: Element-wise multiplication (*) creates interaction terms
    
    # Example interactions to implement:
    # - time * complexity (complex tasks at suspicious times)
    # - location * (1 / (witnesses + 1)) (location effect stronger with fewer witnesses)
    # - time * location (location preferences vary by time)
    
    pass

class EnhancedAccidentDetector(nn.Module):
    """A more sophisticated detector that understands feature interactions."""
    
    def __init__(self, input_features: int = 7):  # 4 original + 3 interactions
        super(EnhancedAccidentDetector, self).__init__()
        # TODO: Create linear layer for enhanced features
        # TODO: Add sigmoid activation
        pass
    
    def forward(self, features: torch.Tensor) -> torch.Tensor:
        # TODO: Implement forward pass with feature interactions
        pass

# TRIAL: Train with interaction features
# SUCCESS: Achieve >80% accuracy with better understanding of feature combinations

### Extension 2: He-Ao-World's Seasonal Accident Patterns
*"Oh dear! I seem to have different accident rates during different temple seasons..."*

*He-Ao-World shuffles over with a mop, looking particularly apologetic*

"I've been thinking about my... clumsiness patterns. Master Pai-Torch mentioned that even accidents follow cycles - spring cleaning season sees more 'mishaps' with dust scrolls, autumn harvest brings more 'spills' near the grain storage, winter meditation requires more 'interruptions' of silent contemplation. Perhaps your detector should understand these seasonal rhythms too?"

**NEW CONCEPTS**: Cyclical features, sine/cosine encoding, temporal patterns  
**DIFFICULTY**: +25% (still Dan 1, but with temporal encoding)

In [ ]:
def add_cyclical_time_features(features: torch.Tensor) -> torch.Tensor:
    """
    Encode time of day cyclically so that 23:00 and 1:00 are close together.
    
    Uses sine and cosine encoding to represent circular time.
    
    Returns:
        Features with added cyclical time encoding [sin_time, cos_time]
    """
    # TODO: Convert hour (0-23) to cyclical representation
    # Hint: sin(2π * hour / 24) and cos(2π * hour / 24)
    # Hint: This makes midnight (0) and midnight (24) the same point
    
    time_of_day = features[:, 0]  # Extract time feature
    
    # TODO: Create sine and cosine encodings
    sin_time = None  # torch.sin(2 * torch.pi * time_of_day / 24)
    cos_time = None  # torch.cos(2 * torch.pi * time_of_day / 24)
    
    # TODO: Combine with original features
    enhanced_features = None
    
    return enhanced_features

def generate_seasonal_accident_data(n_observations: int = 300) -> Tuple[torch.Tensor, torch.Tensor]:
    """
    Generate accident data with stronger cyclical time patterns.
    
    Returns:
        Tuple of (cyclical_features, is_suspicious)
    """
    # TODO: Generate base data and add cyclical time encoding
    # TODO: Modify suspicion patterns to be stronger during certain time cycles
    pass

# TRIAL: Train detector with cyclical time understanding
# SUCCESS: Better performance on time-sensitive patterns, understand cyclical encoding

### Extension 3: Master Pai-Torch's Confidence Calibration
*"Young grasshopper, knowing what you don't know is as important as knowing what you do."*

*Master Pai-Torch sits in quiet contemplation before speaking*

"I observe that your detector speaks with certainty about every incident. But true wisdom lies in understanding confidence. When your model predicts 0.51 probability, is it barely suspicious or clearly suspicious? Learn to calibrate your confidence - predict not just the outcome, but how certain you should be about that prediction."

**NEW CONCEPTS**: Prediction confidence, probability calibration, uncertainty quantification  
**DIFFICULTY**: +35% (still Dan 1, but with confidence analysis)

In [ ]:
def analyze_prediction_confidence(model, features, target, confidence_bins=10):
    """
    Analyze how well-calibrated your model's confidence is.
    
    A well-calibrated model's predictions should match reality:
    - When it predicts 0.7 probability, ~70% should actually be suspicious
    - When it predicts 0.3 probability, ~30% should actually be suspicious
    
    Returns:
        Dictionary with calibration analysis
    """
    model.eval()
    with torch.no_grad():
        predictions = model(features).squeeze()
        
    # TODO: Create confidence bins (0-0.1, 0.1-0.2, ..., 0.9-1.0)
    # TODO: For each bin, calculate:
    #   - Average predicted probability
    #   - Actual fraction of suspicious cases
    #   - Confidence score (how close predicted vs actual)
    
    calibration_results = {}
    
    # TODO: Implement binning and analysis
    for i in range(confidence_bins):
        bin_start = i / confidence_bins
        bin_end = (i + 1) / confidence_bins
        
        # Find predictions in this bin
        in_bin = (predictions >= bin_start) & (predictions < bin_end)
        
        if in_bin.sum() > 0:
            avg_prediction = predictions[in_bin].mean().item()
            actual_fraction = target[in_bin].float().mean().item()
            calibration_results[f'bin_{i}'] = {
                'range': f'{bin_start:.1f}-{bin_end:.1f}',
                'avg_prediction': avg_prediction,
                'actual_fraction': actual_fraction,
                'count': in_bin.sum().item()
            }
    
    return calibration_results

def visualize_confidence_calibration(calibration_results):
    """Plot the calibration curve to see how well-calibrated your model is."""
    # TODO: Create a plot showing predicted vs actual probabilities
    # TODO: Add a diagonal line showing perfect calibration
    # TODO: Show how far your model deviates from perfect calibration
    pass

# TRIAL: Analyze and improve your model's confidence calibration
# SUCCESS: Understand the difference between accuracy and calibration
# MASTERY: Know when your model is overconfident vs underconfident

### Extension 4: Suki's Behavioral Correlation Analysis
*"Meow meow purr... meow." (Translation: "The temple cat sees patterns within patterns.")*

*Suki sits regally atop a stack of scrolls, tail twitching with mathematical precision*

*Master Pai-Torch translates Suki's wisdom: "The sacred cat has observed that He-Ao-World's accidents often correlate with her own behaviors. When Suki sits in doorways, accidents increase near passages. When she naps in the sacred hall, ceremonies get 'accidentally' interrupted. Your detector should learn these cross-pattern correlations to achieve true mastery."*

**NEW CONCEPTS**: Cross-feature correlations, feature importance analysis, interpretable ML  
**DIFFICULTY**: +45% (still Dan 1, but with advanced feature analysis)

In [ ]:
def analyze_feature_importance(model, features, target, feature_names):
    """
    Understand which features your model considers most important.
    
    Uses weight magnitude and gradient-based importance.
    
    Returns:
        Dictionary with importance scores for each feature
    """
    model.eval()
    
    # Method 1: Weight magnitude importance
    weights = model.linear.weight.data.abs().squeeze()
    weight_importance = weights / weights.sum()
    
    # Method 2: Gradient-based importance
    features.requires_grad_(True)
    predictions = model(features)
    loss = nn.BCELoss()(predictions, target)
    
    # TODO: Calculate gradients with respect to input features
    gradients = torch.autograd.grad(loss, features, create_graph=True)[0]
    gradient_importance = gradients.abs().mean(0)
    gradient_importance = gradient_importance / gradient_importance.sum()
    
    # Combine importance measures
    importance_results = {}
    for i, name in enumerate(feature_names):
        importance_results[name] = {
            'weight_importance': weight_importance[i].item(),
            'gradient_importance': gradient_importance[i].item(),
            'combined_importance': (weight_importance[i] + gradient_importance[i]).item() / 2
        }
    
    return importance_results

def feature_ablation_study(model, features, target, feature_names):
    """
    Test how much each feature contributes by removing it and measuring performance drop.
    
    Returns:
        Dictionary showing performance impact of removing each feature
    """
    model.eval()
    
    # Baseline performance with all features
    with torch.no_grad():
        baseline_pred = model(features)
        baseline_acc = ((baseline_pred > 0.5).float() == target).float().mean().item()
    
    ablation_results = {}
    
    # TODO: For each feature, set it to zero and measure performance drop
    for i, feature_name in enumerate(feature_names):
        # Create modified features with one feature zeroed out
        modified_features = features.clone()
        modified_features[:, i] = 0  # Remove this feature
        
        with torch.no_grad():
            modified_pred = model(modified_features)
            modified_acc = ((modified_pred > 0.5).float() == target).float().mean().item()
        
        performance_drop = baseline_acc - modified_acc
        ablation_results[feature_name] = {
            'performance_drop': performance_drop,
            'relative_importance': performance_drop / baseline_acc if baseline_acc > 0 else 0
        }
    
    return ablation_results

def visualize_feature_analysis(importance_results, ablation_results, feature_names):
    """Create comprehensive visualization of feature importance."""
    # TODO: Create bar plots showing:
    # - Weight-based importance
    # - Gradient-based importance  
    # - Ablation study results
    # - Combined importance ranking
    pass

# TRIAL: Understand which features matter most for detecting suspicious timing
# SUCCESS: Identify the most predictive features and explain model behavior
# MASTERY: Use feature analysis to improve model interpretability and trust

## 🔥 CORRECTING YOUR FORM: A STANCE IMBALANCE

*Master Pai-Torch observes your training ritual with careful attention. "Young grasshopper, I sense disturbance in your gradient flow. Your eager mind has rushed ahead of your disciplined form, creating chaos where there should be harmony."*

*A previous disciple left this flawed training ritual. The pattern detection wavers and fails - can you restore proper technique?*

In [ ]:
# 🚨 FLAWED TRAINING RITUAL - RESTORE THE BALANCE!

def broken_pattern_training(model, features, target, epochs=1000):
    """This training stance has lost its balance - can you spot the errors? 🥋"""
    
    criterion = nn.BCELoss()
    optimizer = optim.SGD(model.parameters(), lr=0.01)
    
    losses = []
    
    for epoch in range(epochs):
        # Forward pass
        predictions = model(features)
        
        # Calculate loss
        loss = criterion(predictions, target)
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        losses.append(loss.item())
        
        if epoch % 200 == 0:
            print(f'Epoch {epoch}: Loss = {loss.item():.4f}')
    
    return losses

# DEBUGGING CHALLENGE: Spot the critical errors in this training ritual!
# 
# HINTS FROM MASTER PAI-TORCH:
# 🔮 "The gradient spirits accumulate their whispers from ages past..."
# 🔮 "Without proper cleansing, old wisdom corrupts new learning..."
# 🔮 "Each cycle must begin with a clear mind, free from previous thoughts..."
# 
# ERROR COUNT: 1 critical error (will prevent proper learning)
# 
# MASTER'S WISDOM: "The undisciplined mind carries forward its confusion,
#                   just as the uncleansed gradient carries forward its misdirection."

## 🎭 THE MASTER'S FINAL WISDOM

*As you complete your training, Master Ao-Tougrad emerges fully from the shadows for the first time*

"Grasshopper, you have learned to see beyond the obvious. He-Ao-World's 'accidents' are indeed a masterful dance of timing, location, and opportunity. But remember this deeper truth: in neural networks, as in life, multiple signals combine to reveal hidden patterns. No single feature tells the complete story."

*He-Ao-World shuffles by and winks conspiratorially*

"Your multi-variable classifier has learned what the ancient masters knew: truth emerges from the convergence of many observations, not from any single clue. This sacred principle will serve you well as you advance through the temple's deeper mysteries."

**Sacred Principles Mastered:**
- Multi-dimensional feature processing with PyTorch tensors
- Binary classification with multiple input variables
- Proper gradient management in classification training
- Feature interaction and importance analysis
- Model interpretability and confidence calibration

**Next Temple Lesson**: Continue your journey with more advanced classification techniques, or explore the mysteries of regularization in Dan 2 training.